# Import and Read data


In [1]:
import sys
sys.path.append('.')  # Add current directory to path
from build_graph_and_train import *
import torch

/opt/conda/lib/python3.11/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
partition = 478

In [3]:
df = pd.read_csv(f"../../../data/top30groups/LongLatCombined/combined/combined{partition}.csv")

In [4]:
from sklearn.preprocessing import StandardScaler

# Columns to exclude from scaling
exclude_cols = ['gname', 'longitude', 'latitude']

# Columns to scale
scale_cols = [col for col in df.columns if col not in exclude_cols]

# Scale only selected columns
scaler = StandardScaler()
df[scale_cols] = scaler.fit_transform(df[scale_cols])

In [5]:
import os 
if not os.path.isdir(f"Results{partition}"):
    os.mkdir(f"Results{partition}")

# Create longlat feature

In [6]:
geodata = ['longitude', 'latitude']
combined_geo = df.copy()
combined_geo['longlat'] = list(zip(df['longitude'], df['latitude']))
combined_geo = combined_geo.drop(columns=geodata)

In [7]:
import ast

def to_tuple_if_needed(val):
    if isinstance(val, str):
        return ast.literal_eval(val)
    return val  # already a tuple

combined_geo['longlat'] = combined_geo['longlat'].apply(to_tuple_if_needed)

# Weapon type prediction

In [8]:
torch.cuda.empty_cache()


In [9]:
label_index = {g: i for i, g in enumerate(sorted(df['gname'].unique()))}
continuous_cols = ['weaptype1', 'nkill', 'targtype1', 'attacktype1']
y_preds = []
y_trues = []
logs = []

# Default config (from previous best)
default_args = {
    'partition': f"gtd{partition}",
    'embed_dim': 16,
    'lr': 0.001,
    'epochs': 1000,
    'feat_dropout': 0,
    'n_tree': 80,
    'tree_depth': 10,
    'tree_feature_rate': 0.5,
    'n_class': len(label_index)
}

for col in continuous_cols:
    print(f"\nTraining model for {col} prediction...")

    data, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, row_to_node_index, index_to_label = build_graph_data(
        combined_geo, label_index, continuous_col=col)

    # Train using default parameters
    best_acc, best_epoch, best_precision, best_recall, best_f1, y_pred_decoded, y_true_decoded, \
    best_precision_micro, best_recall_micro, best_f1_micro, best_precision_macro, best_recall_macro, best_f1_macro, \
    roc_auc_weighted, roc_auc_micro, roc_auc_macro, epoch_logs = train_joint(
        data, data.edge_index, y_gcn, y_nrf, non_geo_features, train_mask, test_mask,
        default_args, row_to_node_index, index_to_label, verbose=True)

    y_preds.append(y_pred_decoded)
    y_trues.append(y_true_decoded)
    logs.append(epoch_logs)

    # Save performance metrics
    results_path = f"Results{partition}/Results_{col}_prediction"
    with open(results_path, "w") as f:
        f.write(f"Best acc: {best_acc:.4f} at epoch {best_epoch} for {col} prediction\n")
        f.write(f"Weighted Precision: {best_precision:.4f}, Recall: {best_recall:.4f}, F1: {best_f1:.4f}\n")
        f.write(f"Macro Precision: {best_precision_macro:.4f}, Recall: {best_recall_macro:.4f}, F1: {best_f1_macro:.4f}\n")
        f.write(f"Micro Precision: {best_precision_micro:.4f}, Recall: {best_recall_micro:.4f}, F1: {best_f1_micro:.4f}\n")
        f.write(f"AUROC Weighted: {roc_auc_weighted:.4f}, Micro: {roc_auc_micro:.4f}, Macro: {roc_auc_macro:.4f}\n")

    # Save epoch timings
    log_path = f"Results{partition}/epoch_logs_{col}_prediction"
    with open(log_path, "w") as f:
        f.write('\n'.join(f"{x:.4f}" for x in epoch_logs))



Training model for weaptype1 prediction...


Training epochs:   0%|          | 1/1000 [00:01<20:06,  1.21s/it]

Epoch 01 | GCN MSE Loss: 1439.0708 | NRF Loss: 3.3992 | JOINT Loss: 1442.4701 | NRF Acc: 0.0668


Training epochs:   0%|          | 2/1000 [00:01<14:45,  1.13it/s]

Epoch 02 | GCN MSE Loss: 1403.9645 | NRF Loss: 3.3869 | JOINT Loss: 1407.3513 | NRF Acc: 0.0668


...

Epoch 998 | GCN MSE Loss: 1.5437 | NRF Loss: 1.8902 | JOINT Loss: 3.4339 | NRF Acc: 0.8050


Training epochs: 100%|█████████▉| 999/1000 [14:41<00:00,  1.24it/s]

Epoch 999 | GCN MSE Loss: 1.5432 | NRF Loss: 1.8893 | JOINT Loss: 3.4326 | NRF Acc: 0.8043


Training epochs: 100%|██████████| 1000/1000 [14:42<00:00,  1.13it/s]

Epoch 1000 | GCN MSE Loss: 1.5427 | NRF Loss: 1.8885 | JOINT Loss: 3.4313 | NRF Acc: 0.8061
Best acc/epoch: 0.8098671436309814, epoch 985


In [10]:
from sklearn.metrics import classification_report

print(classification_report(y_pred_decoded, y_true_decoded))

                                                  precision    recall  f1-score   support

                          Abu Sayyaf Group (ASG)       0.99      0.94      0.96       305
        African National Congress (South Africa)       1.00      1.00      1.00       352
                                Al-Qaida in Iraq       0.92      0.72      0.81       539
        Al-Qaida in the Arabian Peninsula (AQAP)       0.97      0.69      0.80       504
                                      Al-Shabaab       0.96      0.95      0.96       340
             Basque Fatherland and Freedom (ETA)       0.99      0.91      0.94       407
                                      Boko Haram       0.83      0.98      0.90       237
  Communist Party of India - Maoist (CPI-Maoist)       0.54      0.84      0.66       153
       Corsican National Liberation Front (FLNC)       1.00      0.96      0.98       439
                       Donetsk People's Republic       1.00      0.99      1.00       381
Farabundo

/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [11]:
def plot_confusion_matrix(y_true, y_pred, labels, continuous_col):
    from sklearn.metrics import confusion_matrix
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np

    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_normalized = cm.astype('float') / cm.sum(axis=1, keepdims=True)

    plt.figure(figsize=(18, 16))
    sns.heatmap(cm_normalized,
                annot=True,
                fmt=".2f",
                xticklabels=labels,
                yticklabels=labels,
                cmap="viridis",
                square=True,
                linewidths=0.5,
                cbar_kws={"shrink": 0.8})

    plt.title(f"Normalized Confusion Matrix", fontsize=18)
    plt.xlabel("Predicted Label", fontsize=14)
    plt.ylabel("True Label", fontsize=14)
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()

    # Save the figure
    save_path = f"Results{partition}/cm_{partition}_{continuous_col}.png"
    plt.savefig(save_path, dpi=300)
    plt.close()

    print(f"Saved confusion matrix for partition {partition} to {save_path}")


In [12]:
for i in range(len(continuous_cols)):
    plot_confusion_matrix(y_preds[i], y_trues[i], sorted(df['gname'].unique()), continuous_cols[i])

/tmp/ipykernel_13666/4012392659.py:8: RuntimeWarning: invalid value encountered in divide
  cm_normalized = cm.astype('float') / cm.sum(axis=1, keepdims=True)


Saved confusion matrix for partition 478 to Results478/cm_478_weaptype1.png


/tmp/ipykernel_13666/4012392659.py:8: RuntimeWarning: invalid value encountered in divide
  cm_normalized = cm.astype('float') / cm.sum(axis=1, keepdims=True)


Saved confusion matrix for partition 478 to Results478/cm_478_nkill.png


/tmp/ipykernel_13666/4012392659.py:8: RuntimeWarning: invalid value encountered in divide
  cm_normalized = cm.astype('float') / cm.sum(axis=1, keepdims=True)


Saved confusion matrix for partition 478 to Results478/cm_478_targtype1.png


/tmp/ipykernel_13666/4012392659.py:8: RuntimeWarning: invalid value encountered in divide
  cm_normalized = cm.astype('float') / cm.sum(axis=1, keepdims=True)


Saved confusion matrix for partition 478 to Results478/cm_478_attacktype1.png
